In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
import numpy as np
from particle import literals as lp
import DarkNews as dn
from DarkNews import model
from DarkNews import const
from DarkNews import GenLauncher

In [57]:
lp.nu_mu_bar

<Particle: name="nu(mu)~", pdgid=-14, mass=None>

In [4]:
MODEL_KWARGS = {'HNLtype': 'dirac', 'UD4': 1.0, 'alphaD': 0.25, 'Umu4': np.sqrt(9e-7), 'epsilon': np.sqrt(2e-10/const.alphaQED)}

gen = GenLauncher(mzprime=0.2, m4=0.4, decay_product='mu+mu-', neval=1000, exp="miniboone_fhc", loglevel='DEBUG', seed=42, **MODEL_KWARGS)
df_heavy = gen.run()


---------------------------------------------------------
   ______           _        _   _                     
   |  _  \         | |      | \ | |                    
   | | | |__ _ _ __| | __   |  \| | _____      _____   
   | | | / _  | ___| |/ /   | .   |/ _ \ \ /\ / / __|  
   | |/ / (_| | |  |   <    | |\  |  __/\ V  V /\__ \  
   |___/ \__,_|_|  |_|\_\   \_| \_/\___| \_/\_/ |___/  

---------------------------------------------------------
Model:
	1 dirac heavy neutrino(s).

---------------------------------------------------------
Experiment:
	MiniBooNE_FHC
	fluxfile loaded: ../fluxes/MiniBooNE_FHC.dat
	POT: 1.875e+21
	nuclear targets: ['C12', 'H1']
	fiducial mass: [701.1428571428571, 116.85714285714286] tonnes

Directory tree for this run already exists. Overriding it.
Creating instances of MC cases:
Created an MC instance of nu(mu) C12 --> N4  C12 --> nu_light mu+ mu- C12.
Created an MC instance of nu(mu) proton_in_C12 --> N4  proton_in_C12 --> nu_light mu+ mu- proton_in_C1

In [26]:
# generate a specific set of events to be tested in a session
def SM_gen():
    gen = GenLauncher(gD=0.0, Umu4=1e-3, epsilon=0.0, m4=0.01, loglevel='ERROR', neval=1000, seed=42)
    return gen.run()

def light_DP_gen_all_outputs():

    gen = GenLauncher(mzprime=0.03, m4=0.420, neval=1000, exp="miniboone_fhc", loglevel='INFO',
                        parquet=True, numpy=True, hepevt=True, sparse=True, print_to_float32=True, **MODEL_KWARGS)

    return gen.run(loglevel="INFO")

def gen_simplest_benchmarks():

    gen = GenLauncher(mzprime=0.03, m4=0.420, neval=1000, exp="miniboone_fhc", loglevel='ERROR', seed=42, **MODEL_KWARGS)
    df_light = gen.run()

    gen = GenLauncher(mzprime=1.25, m4=0.150, neval=1000, exp="miniboone_fhc", loglevel='ERROR', seed=42, **MODEL_KWARGS)
    df_heavy = gen.run()

    gen = GenLauncher(mu_tr_mu4=1e-6, m4=0.150, epsilon=0.0, gD=0.0, decay_product='e+e-', Umu4=0.0,
                    neval=1000, exp="miniboone_fhc", loglevel='ERROR', seed=42)
    df_TMM = gen.run()

    return df_light, df_heavy, df_TMM

def gen_other_final_states():

    gen = GenLauncher(mzprime=0.3, m4=0.5, decay_product='mu+mu-', neval=1000, exp="miniboone_fhc", loglevel='ERROR', seed=42, **MODEL_KWARGS)
    df_light = gen.run()

    gen = GenLauncher(mzprime=1.25, m4=0.5, decay_product='mu+mu-', neval=1000, exp="miniboone_fhc", loglevel='ERROR', seed=42, **MODEL_KWARGS)
    df_heavy = gen.run()

    gen = GenLauncher(mu_tr_mu4=1e-6, m4=0.5, decay_product='mu+mu-', Umu4=0.0, epsilon=0.0, gD=0.0,
                        neval=1000, HNLtype="dirac", exp="miniboone_fhc", loglevel='ERROR', seed=42)
    df_TMM_mumu = gen.run()

    gen = GenLauncher(mu_tr_mu4=1e-6, m4=0.5, decay_product='photon', Umu4=0.0, epsilon=0.0, gD=0.0,
                        neval=1000, HNLtype="dirac", exp="miniboone_fhc", loglevel='ERROR', seed=42)
    df_TMM_photon = gen.run()

    return df_light, df_heavy, df_TMM_mumu, df_TMM_photon





In [49]:
df = SM_gen()
df.w_event_rate.sum()

0.038374333391819795

In [27]:
df_light,df_heavy,df_TMM = gen_simplest_benchmarks()

print(df_light.w_event_rate.sum())
print(df_heavy.w_event_rate.sum())
print(df_TMM.w_event_rate.sum())

13504.557608335577
6.384879373980024
52283.95934407082


In [30]:
df_light, df_heavy, df_TMM_mumu, df_TMM_photon = gen_other_final_states()

print(df_light.w_event_rate.sum())
print(df_heavy.w_event_rate.sum())
print(df_TMM_mumu.w_event_rate.sum())
print(df_TMM_photon.w_event_rate.sum())

205.54654591769042
2.4904013871142787
3426.3242951081156
3446.785418216368


-9.464465024579563e-09

In [31]:
assert df_TMM_mumu['P_decay_ell_plus','0'].min() > dn.const.m_mu, 'Mu+ energy smaller than its mass? Not generating for muons?'


AssertionError: Mu+ energy smaller than its mass? Not generating for muons?

In [23]:
df_TMM_mumu['P_decay_ell_plus','0'].min()

0.10643404724482186

In [22]:
assert 'P_decay_photon' in df_TMM_photon.columns

In [20]:
assert 1<0, 'asdas'

AssertionError: asdas

In [14]:
df_TMM_photon.P_decay_photon

,0,1,2,3
0,0.660944,-0.208607,0.013830,0.627008
1,0.252918,0.077756,0.130266,0.202366
2,0.468576,0.076118,0.244791,0.392234
3,0.742639,-0.145130,-0.365141,0.630177
4,0.227800,0.034881,0.192291,0.117049
...,...,...,...,...
5576,0.252618,-0.244388,0.013929,-0.062422
5577,0.256000,-0.150856,-0.081645,0.190033
5578,1.955821,0.201239,-0.054731,1.944671
5579,1.038716,0.087873,-0.057545,1.033392


In [4]:
bsm_model_1 = dn.model.create_3portal_HNL_model(mzprime=1.25, m4=0.1, m5=0.150, Umu4=1e-3, Umu5=2e-3, UD4=0.2, UD5=0.2, epsilon=1e-4)
# bsm_model_2 = dn.model.create_generic_HNL_model()

AttributeError: 'ThreePortalModel' object has no attribute 'HNLtype'

In [107]:
x = bsm_model_1.d_ij
print(np.array2string(x, formatter={'float_kind':lambda x: "%.1E" % x if x !=0 else "0.00000"}))

[[-3.7E-08 0.00000 0.00000 0.00000 0.00000 0.00000]
 [0.00000 9.6E-07 0.00000 2.0E-04 2.0E-04 0.00000]
 [0.00000 0.00000 -3.7E-08 0.00000 0.00000 0.00000]
 [0.00000 2.0E-04 0.00000 4.0E-02 4.0E-02 0.00000]
 [0.00000 2.0E-04 0.00000 4.0E-02 4.0E-02 0.00000]
 [0.00000 0.00000 0.00000 0.00000 0.00000 0.00000]]


In [108]:
print(np.sqrt(np.sum(np.abs(bsm_model_1.d_aj[const.inds_active,3])**2)))
print(np.sqrt(np.sum(np.abs(bsm_model_1.d_ij[const.inds_active,3])**2)))

0.00020784611846974275
0.00019999996265424277


In [109]:
# x = np.vstack((bsm_model_1.d_aj,bsm_model_1.d_ij[3:]))
x = bsm_model_1.d_aj
print(np.array2string(x, formatter={'float_kind':lambda x: "%.1E" % x if x !=0 else "0.00000"}))

[[1.0E+00 0.00000 0.00000 -1.2E-04 -1.2E-04 0.00000]
 [0.00000 1.0E+00 0.00000 -1.2E-04 -1.2E-04 0.00000]
 [0.00000 0.00000 1.0E+00 -1.2E-04 -1.2E-04 0.00000]
 [0.00000 2.0E-04 0.00000 4.0E-02 4.0E-02 0.00000]
 [0.00000 2.0E-04 0.00000 4.0E-02 4.0E-02 0.00000]
 [0.00000 0.00000 0.00000 0.00000 0.00000 0.00000]]


In [116]:
decay_1=dn.processes.FermionDileptonDecay(dn.pdg.neutrino4, dn.pdg.nulight, lp.e_minus, lp.e_plus, bsm_model_1)
ups_case=dn.processes.UpscatteringProcess(dn.pdg.numu, bsm_model_1.neutrino4, dn.nuclear_tools.NuclearTarget("C12"), 'coherent', bsm_model_1, 'conserving')
# decay_2=dn.model.FermionDileptonDecay(dn.pdg.neutrino4, dn.pdg.nulight, lp.e_minus, lp.e_plus, bsm_model_2)

In [117]:
const.gweak/2.0/const.cw*np.abs((1.0-4.0*const.s2w)*self.target.Z-self.target.N)

NameError: name 'self' is not defined

In [124]:
ups_case.Chad/(bsm_model_1.cVproton*6 + bsm_model_1.cVneutron*6)

-1.9999999771191013

In [119]:
ups_case.Vhad/(bsm_model_1.dVproton*6 + bsm_model_1.dVneutron*6)

0.98127287696488

In [110]:
bsm_model = dn.model.create_HNL_model(mzprime=0.1, 
                                            m5=0.150, 
                                            m4 = 0.01, 
                                            epsilon=0.0, 
                                            theta=1e-4, 
                                            s_mu4=1e-3
                                        )

AttributeError: module 'DarkNews.model' has no attribute 'create_HNL_model'